<a href="https://colab.research.google.com/github/Parlipuudja/TwinState/blob/main/Clone_One_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP next steps

1. Pick: one MP, one training policy, one test policy, one model to use.

2. Implement the code for this all, the very first working prediction.

# Model architecture



### At training

Input: policy text, MP votes, speeches.

Model: learns connection between MP votes and their speeches.

### At test/final product

Input: policy text.

Model: predicts position of every MP to policy.

Output: "support / oppose / unclear" + confidence, and three linked quotes each (from past debates), of 1 up to 101 MPs.


### Per each MP whose position is predicted

Input/stored data: speeches, votes

Model: small-size reasoning LLM

Output: "support / oppose / unclear" + confidence, and three linked quotes each (from past debates).

# Steps in the code

### 1. Fetch relevant data from Riigikogu API.

### 2. Prepare data for use by the model.

### 3. Run the model on the data.

### 4. Output some results from the model.

In [ ]:
import json, time, pathlib, requests
from urllib.parse import urljoin

BASE = "https://api.riigikogu.ee"
OUT  = pathlib.Path("riigikogu_snapshot"); OUT.mkdir(exist_ok=True)

def fetch_json(url, params=None):
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# 1) Members (plenary)
members = fetch_json(urljoin(BASE, "/api/plenary-members"))
(pathlib.Path(OUT, "members.json")).write_text(json.dumps(members, ensure_ascii=False, indent=2))

# 2) Votings for a sample day (adjust dates)
params = {"startDate":"2025-05-14", "endDate":"2025-05-14", "lang":"et"}
votings = fetch_json(urljoin(BASE, "/api/votings"), params=params)
(pathlib.Path(OUT, "votings-2025-05-14.json")).write_text(json.dumps(votings, ensure_ascii=False, indent=2))

# 3) Verbatim record HTML by datetime (use known pattern)
def fetch_stenogram(dt="201812061000", lang="et"):
    # examples: et/201812061000 or et/202304191400
    url = f"https://stenogrammid.riigikogu.ee/{lang}/{dt}"
    r = requests.get(url, timeout=30)
    if r.status_code == 200:
        (OUT / f"stenogram-{dt}.html").write_bytes(r.content)
        return url
    return None

for dt in ["201812061000", "202304191400"]:
    print("Saved:", fetch_stenogram(dt))


Saved: https://stenogrammid.riigikogu.ee/et/201812061000
Saved: https://stenogrammid.riigikogu.ee/et/202304191400
